### 1. Configs

In [1]:
# libraries
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
# pandas options
pd.set_option("display.max_columns", None)
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
# garbage collection
import gc
gc.enable()

### 2. Functions

* FUNCTION FOR COUNTING MISSINGS

In [2]:
def count_missings(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum() / data.isnull().count() * 100).sort_values(ascending = False)
    table = pd.concat([total, percent], axis = 1, keys = ["Total", "Percent"])
    table = table[table["Total"] > 0]
    return table

* FUNCTION FOR CONVERTING DATES

In [3]:
def convert_days(data, features, t = 12, rounding = True, replace = False):
    for var in features:
        if replace == True:
            if rounding == True:
                data[var] = round(-data[var]/t)
            else:
                data[var] = -data[var]/t
            data[var][data[var] < 0] = None
        else:
            if rounding == True:
                data["CONVERTED_" + str(var)] = round(-data[var]/t)
            else:
                data["CONVERTED_" + str(var)] = -data[var]/t
            data["CONVERTED_" + str(var)][data["CONVERTED_" + str(var)] < 0] = None
    return data

* FUNCTION FOR CREATING LOGARITHMS

In [4]:
def create_logs(data, features, replace = False):
    for var in features:
        if replace == True:
            data[var] = np.log(data[var].abs() + 1)
        else:
            data["LOG_" + str(var)] = np.log(data[var].abs() + 1)      
    return data

* FUNCTION FOR CREATING FLAGS FOR MISSINGS

In [5]:
def create_null_flags(data, features = None):
    if features == None:
        features = data.columns
    for var in features:
        num_null = data[var].isnull() + 0
        if num_null.sum() > 0:
            data["ISNULL_" + str(var)] = num_null
    return data

* FUNCTION FOR TREATING FACTORS


In [6]:
def treat_factors(data, method = "label"):    
    # label encoding
    if method == "label":
        factors = [f for f in data.columns if data[f].dtype == "object"]
        for var in factors:
            data[var], _ = pd.factorize(data[var])
        
    # dummy encoding
    if method == "dummy":
        data = pd.get_dummies(data, drop_first = True)
    
    # dataset
    return data

* FUNCTION FOR COMPUTING ACCEPT/REJECT RATIOS

In [7]:
def compute_accept_reject_ratio(data, lags = [1, 3, 5]):
    
    # preparations
    dec_prev = data[["SK_ID_CURR", "SK_ID_PREV", "DAYS_DECISION", "NAME_CONTRACT_STATUS"]]
    dec_prev["DAYS_DECISION"] = -dec_prev["DAYS_DECISION"]
    dec_prev = dec_prev.sort_values(by = ["SK_ID_CURR", "DAYS_DECISION"])
    dec_prev = pd.get_dummies(dec_prev)
     
    # compuatation
    for t in lags:
        
        # acceptance ratios
        tmp = dec_prev[["SK_ID_CURR", "NAME_CONTRACT_STATUS_Approved"]].groupby(["SK_ID_CURR"]).head(1)
        tmp = tmp.groupby(["SK_ID_CURR"], as_index = False).mean()
        tmp.columns = ["SK_ID_CURR", "APPROVE_RATIO_" + str(t)]
        data = data.merge(tmp, how = "left", on = "SK_ID_CURR")
        
        # rejection ratios
        tmp = dec_prev[["SK_ID_CURR", "NAME_CONTRACT_STATUS_Refused"]].groupby(["SK_ID_CURR"]).head(1)
        tmp = tmp.groupby(["SK_ID_CURR"], as_index = False).mean()
        tmp.columns = ["SK_ID_CURR", "REJECT_RATIO_" + str(t)]
        data = data.merge(tmp, how = "left", on = "SK_ID_CURR")
        
    # dataset
    return data

* FUNCTION FOR AGGREGATING DATA

In [8]:
def aggregate_data(data, id_var, label = None):
    
    
    ### SEPARATE FEATURES
  
    # display info
    print("- Preparing the dataset...")

    # find factors
    data_factors = [f for f in data.columns if data[f].dtype == "object"]
    
    # partition subsets
    num_data = data[list(set(data.columns) - set(data_factors))]
    fac_data = data[[id_var] + data_factors]
    
    # display info
    num_facs = fac_data.shape[1] - 1
    num_nums = num_data.shape[1] - 1
    print("- Extracted %.0f factors and %.0f numerics..." % (num_facs, num_nums))


    ##### AGGREGATION
 
    # aggregate numerics
    if (num_nums > 0):
        print("- Aggregating numeric features...")
        num_data = num_data.groupby(id_var).agg(["mean", "std", "min", "max"])
        num_data.columns = ["_".join(col).strip() for col in num_data.columns.values]
        num_data = num_data.sort_index()

    # aggregate factors
    if (num_facs > 0):
        print("- Aggregating factor features...")
        fac_data = fac_data.groupby(id_var).agg([("mode",   lambda x: scipy.stats.mode(x)[0][0]),
                                                 ("unique", lambda x: x.nunique())])
        fac_data.columns = ["_".join(col).strip() for col in fac_data.columns.values]
        fac_data = fac_data.sort_index()


    ##### MERGER

    # merge numerics and factors
    if ((num_facs > 0) & (num_nums > 0)):
        agg_data = pd.concat([num_data, fac_data], axis = 1)
    
    # use factors only
    if ((num_facs > 0) & (num_nums == 0)):
        agg_data = fac_data
        
    # use numerics only
    if ((num_facs == 0) & (num_nums > 0)):
        agg_data = num_data
        

    ##### LAST STEPS

    # update labels
    if label != None:
        agg_data.columns = [label + "_" + str(col) for col in agg_data.columns]
    
    # impute zeros for SD
    #stdevs = agg_data.filter(like = "_std").columns
    #for var in stdevs:
    #    agg_data[var].fillna(0, inplace = True)

    # display info
    print("- Final dimensions:", agg_data.shape)
    
    # return dataset
    return agg_data

### 3. Preprocess

#### 3.1. POS cash data

In [9]:
poca = pd.read_csv('C:\\Users\\Dell V3400\\Downloads\\visualization\\dseb63_final_project_DP_dataset\\dseb63_POS_CASH_balance.csv')

In [10]:
poca.head()

,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,SK_ID_CURR
0,1803195,-31,48.0,45.0,Active,0,0,185279
1,1803195,-17,48.0,31.0,Active,0,0,185279
2,1803195,-21,48.0,35.0,Active,0,0,185279
3,1803195,-8,48.0,21.0,Active,0,0,185279
4,1803195,-4,48.0,17.0,Active,0,0,185279


* FEATURE ENGINEERING

In [11]:
# installments percentage
poca["INSTALLMENTS_PERCENT"] = poca["CNT_INSTALMENT_FUTURE"] / poca["CNT_INSTALMENT"]

In [12]:
# dummy encodnig for factors
poca = pd.get_dummies(poca, drop_first = True)

In [13]:
# count missings
nas = count_missings(poca)
nas.head()

,Total,Percent
INSTALLMENTS_PERCENT,21963,0.257076
CNT_INSTALMENT_FUTURE,21878,0.256081
CNT_INSTALMENT,21863,0.255906


In [14]:
poca.head()

,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,SK_ID_CURR,INSTALLMENTS_PERCENT,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA
0,1803195,-31,48.0,45.0,0,0,185279,0.937500,False,False,False,False,False,False,False,False
1,1803195,-17,48.0,31.0,0,0,185279,0.645833,False,False,False,False,False,False,False,False
2,1803195,-21,48.0,35.0,0,0,185279,0.729167,False,False,False,False,False,False,False,False
3,1803195,-8,48.0,21.0,0,0,185279,0.437500,False,False,False,False,False,False,False,False
4,1803195,-4,48.0,17.0,0,0,185279,0.354167,False,False,False,False,False,False,False,False


* AGGREGATIONS

In [15]:
# count months
cnt_mon = poca[["SK_ID_PREV", "MONTHS_BALANCE"]].groupby("SK_ID_PREV").count()
del poca["MONTHS_BALANCE"]

In [16]:
cnt_mon.head()

,MONTHS_BALANCE
SK_ID_PREV,
1000001,3
1000003,4
1000004,8
1000005,11
1000007,5


In [17]:
pos_agg_list = {'CNT_INSTALMENT':['min','max'],
            'CNT_INSTALMENT_FUTURE':['min','max'],
            'SK_DPD':['max','mean'],
            'SK_DPD_DEF':['max','mean'],
            'NAME_CONTRACT_STATUS_Amortized debt':'sum',
            'NAME_CONTRACT_STATUS_Amortized debt':'sum',
            'NAME_CONTRACT_STATUS_Approved':'sum',
            'NAME_CONTRACT_STATUS_Canceled':'sum',
            'NAME_CONTRACT_STATUS_Completed':'sum',
            'NAME_CONTRACT_STATUS_Demand':'sum',
            'NAME_CONTRACT_STATUS_Returned to the store':'sum',
            'NAME_CONTRACT_STATUS_Signed':'sum',
            'NAME_CONTRACT_STATUS_XNA':'sum'}

In [18]:
poca.head()

,SK_ID_PREV,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,SK_ID_CURR,INSTALLMENTS_PERCENT,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA
0,1803195,48.0,45.0,0,0,185279,0.937500,False,False,False,False,False,False,False,False
1,1803195,48.0,31.0,0,0,185279,0.645833,False,False,False,False,False,False,False,False
2,1803195,48.0,35.0,0,0,185279,0.729167,False,False,False,False,False,False,False,False
3,1803195,48.0,21.0,0,0,185279,0.437500,False,False,False,False,False,False,False,False
4,1803195,48.0,17.0,0,0,185279,0.354167,False,False,False,False,False,False,False,False


In [19]:
pos_agg = poca.groupby('SK_ID_PREV').agg(pos_agg_list)


In [20]:
pos_agg.head()

CNT_INSTALMENT       CNT_INSTALMENT_FUTURE       SK_DPD       \
                      min   max                   min   max    max mean   
SK_ID_PREV                                                                
1000001               2.0  12.0                   0.0  12.0      0  0.0   
1000003              12.0  12.0                   9.0  12.0      0  0.0   
1000004               7.0  10.0                   0.0  10.0      0  0.0   
1000005              10.0  10.0                   0.0  10.0      0  0.0   
1000007               6.0   6.0                   2.0   6.0      0  0.0   

           SK_DPD_DEF      NAME_CONTRACT_STATUS_Amortized debt  \
                  max mean                                 sum   
SK_ID_PREV                                                       
1000001             0  0.0                                   0   
1000003             0  0.0                                   0   
1000004             0  0.0                                   0   
1000005             0  0.0                                   0   
1000007             0  0.0                                   0   

           NAME_CONTRACT_STATUS_Approved NAME_CONTRACT_STATUS_Canceled  \
                                     sum                           sum   
SK_ID_PREV                                                               
1000001                                0                             0   
1000003                                0                             0   
1000004                                0                             0   
1000005                                0                             0   
1000007                                0                             0   

           NAME_CONTRACT_STATUS_Completed NAME_CONTRACT_STATUS_Demand  \
                                      sum                         sum   
SK_ID_PREV                                                              
1000001                                 1                           0   
1000003                                 0                           0   
1000004                                 1                           0   
1000005                                 1                           0   
1000007                                 0                           0   

           NAME_CONTRACT_STATUS_Returned to the store  \
                                                  sum   
SK_ID_PREV                                              
1000001                                             0   
1000003                                             0   
1000004                                             0   
1000005                                             0   
1000007                                             0   

           NAME_CONTRACT_STATUS_Signed NAME_CONTRACT_STATUS_XNA  
                                   sum                      sum  
SK_ID_PREV                                                       
1000001                              0                        0  
1000003                              0                        0  
1000004                              0                        0  
1000005                              0                        0  
1000007                              0                        0

In [21]:
pos_agg.columns= pd.Index([e[0] + '_' + e[1].upper() for e in pos_agg.columns.tolist()])


In [22]:
pos_agg.head()

,CNT_INSTALMENT_MIN,CNT_INSTALMENT_MAX,CNT_INSTALMENT_FUTURE_MIN,CNT_INSTALMENT_FUTURE_MAX,SK_DPD_MAX,SK_DPD_MEAN,SK_DPD_DEF_MAX,SK_DPD_DEF_MEAN,NAME_CONTRACT_STATUS_Amortized debt_SUM,NAME_CONTRACT_STATUS_Approved_SUM,NAME_CONTRACT_STATUS_Canceled_SUM,NAME_CONTRACT_STATUS_Completed_SUM,NAME_CONTRACT_STATUS_Demand_SUM,NAME_CONTRACT_STATUS_Returned to the store_SUM,NAME_CONTRACT_STATUS_Signed_SUM,NAME_CONTRACT_STATUS_XNA_SUM
SK_ID_PREV,,,,,,,,,,,,,,,,
1000001,2.0,12.0,0.0,12.0,0,0.0,0,0.0,0,0,0,1,0,0,0,0
1000003,12.0,12.0,9.0,12.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0
1000004,7.0,10.0,0.0,10.0,0,0.0,0,0.0,0,0,0,1,0,0,0,0
1000005,10.0,10.0,0.0,10.0,0,0.0,0,0.0,0,0,0,1,0,0,0,0
1000007,6.0,6.0,2.0,6.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0


In [23]:
pos_agg['NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME']= (pos_agg['CNT_INSTALMENT_FUTURE_MIN']==0) & (pos_agg['NAME_CONTRACT_STATUS_Completed_SUM']==0)


In [24]:
pos_agg['NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME']=pos_agg['NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME'].astype(int)


In [25]:
pos_agg.head()

,CNT_INSTALMENT_MIN,CNT_INSTALMENT_MAX,CNT_INSTALMENT_FUTURE_MIN,CNT_INSTALMENT_FUTURE_MAX,SK_DPD_MAX,SK_DPD_MEAN,SK_DPD_DEF_MAX,SK_DPD_DEF_MEAN,NAME_CONTRACT_STATUS_Amortized debt_SUM,NAME_CONTRACT_STATUS_Approved_SUM,NAME_CONTRACT_STATUS_Canceled_SUM,NAME_CONTRACT_STATUS_Completed_SUM,NAME_CONTRACT_STATUS_Demand_SUM,NAME_CONTRACT_STATUS_Returned to the store_SUM,NAME_CONTRACT_STATUS_Signed_SUM,NAME_CONTRACT_STATUS_XNA_SUM,NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME
SK_ID_PREV,,,,,,,,,,,,,,,,,
1000001,2.0,12.0,0.0,12.0,0,0.0,0,0.0,0,0,0,1,0,0,0,0,0
1000003,12.0,12.0,9.0,12.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0
1000004,7.0,10.0,0.0,10.0,0,0.0,0,0.0,0,0,0,1,0,0,0,0,0
1000005,10.0,10.0,0.0,10.0,0,0.0,0,0.0,0,0,0,1,0,0,0,0,0
1000007,6.0,6.0,2.0,6.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0


In [26]:
# count missings
nas = count_missings(pos_agg)
nas.head()

,Total,Percent
CNT_INSTALMENT_MIN,774,0.096709
CNT_INSTALMENT_FUTURE_MIN,774,0.096709
CNT_INSTALMENT_FUTURE_MAX,774,0.096709
CNT_INSTALMENT_MAX,774,0.096709


In [27]:
pos_agg.to_csv('C:\\Users\\Dell V3400\\Downloads\\visualization\\dseb63_final_project_DP_dataset\\pos_agg_demo.csv')

#### 3.2. installments

In [28]:
inst = pd.read_csv('C:\\Users\\Dell V3400\\Downloads\\visualization\\dseb63_final_project_DP_dataset\\dseb63_installments_payments.csv')

In [29]:
inst

,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR
0,1054186,1.0,6,-1180.0,-1187.0,6948.360,6948.360,147397.0
1,2452854,1.0,21,-546.0,-552.0,11302.605,11302.605,147397.0
2,1054186,1.0,2,-1300.0,-1307.0,6948.360,6948.360,147397.0
3,1682318,1.0,2,-240.0,-243.0,7374.510,7374.510,147397.0
4,2452854,1.0,10,-876.0,-882.0,11302.605,11302.605,147397.0
...,...,...,...,...,...,...,...,...
7744753,2192667,1.0,6,-2352.0,-2352.0,5322.240,5322.240,21216.0
7744754,2208281,1.0,4,-452.0,-466.0,63195.435,63195.435,21216.0
7744755,2657771,0.0,3,-2907.0,-2932.0,3375.000,3375.000,21216.0
7744756,2657771,0.0,47,-1871.0,-1871.0,4915.890,4915.890,21216.0


* FEATURE ENGINEERING

In [30]:
# days past due and days before due (no negative values)
inst['DPD'] = inst['DAYS_ENTRY_PAYMENT'] - inst['DAYS_INSTALMENT']
inst['DBD'] = inst['DAYS_INSTALMENT'] - inst['DAYS_ENTRY_PAYMENT']
inst['DPD'] = inst['DPD'].apply(lambda x: x if x > 0 else 0)
inst['DBD'] = inst['DBD'].apply(lambda x: x if x > 0 else 0)

# percentage and difference paid in each installment 
#inst['PAYMENT_PERC'] = inst['AMT_PAYMENT'] / inst['AMT_INSTALMENT']
#inst['PAYMENT_DIFF'] = inst['AMT_INSTALMENT'] - inst['AMT_PAYMENT']

In [31]:
# dummy encodnig for factors
inst = pd.get_dummies(inst, drop_first = True)

In [32]:
# count missings
nas = count_missings(inst)
nas.head()

,Total,Percent
DAYS_ENTRY_PAYMENT,1639,0.021163
AMT_PAYMENT,1639,0.021163
AMT_INSTALMENT,1,0.000013
SK_ID_CURR,1,0.000013


* AGGREGATIONS

In [33]:
inst.columns

Index(['SK_ID_PREV', 'NUM_INSTALMENT_VERSION', 'NUM_INSTALMENT_NUMBER',
       'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT', 'AMT_INSTALMENT',
       'AMT_PAYMENT', 'SK_ID_CURR', 'DPD', 'DBD'],
      dtype='object')

In [34]:
agg_list = {'NUM_INSTALMENT_VERSION':['nunique'],
               'NUM_INSTALMENT_NUMBER':'max',
               'DAYS_INSTALMENT':['min','max'],
               'DAYS_ENTRY_PAYMENT':['min','max'],
               'AMT_INSTALMENT':['min','max','sum','mean'],
               'AMT_PAYMENT':['min','max','sum','mean'],
               'DPD':'mean',
               'DBD':'mean'}

In [35]:
ins_agg = inst.groupby('SK_ID_PREV').agg(agg_list)

In [36]:
ins_agg

NUM_INSTALMENT_VERSION NUM_INSTALMENT_NUMBER DAYS_INSTALMENT  \
                          nunique                   max             min   
SK_ID_PREV                                                                
1000001                         2                     2          -268.0   
1000003                         1                     3           -94.0   
1000004                         2                     7          -862.0   
1000005                         1                    10         -1688.0   
1000007                         1                     5          -123.0   
...                           ...                   ...             ...   
2843490                         1                     4         -2706.0   
2843491                         1                    10          -274.0   
2843492                         1                    12          -668.0   
2843494                         2                     2          -781.0   
2843495                         2                     7          -439.0   

                   DAYS_ENTRY_PAYMENT         AMT_INSTALMENT              \
               max                min     max            min         max   
SK_ID_PREV                                                                 
1000001     -238.0             -294.0  -244.0       6404.310   62039.115   
1000003      -34.0             -108.0   -49.0       4951.350    4951.350   
1000004     -682.0             -881.0  -695.0       3391.110   13176.495   
1000005    -1418.0            -1687.0 -1433.0      14599.260   14713.605   
1000007       -3.0             -143.0   -10.0      11246.805   11246.805   
...            ...                ...     ...            ...         ...   
2843490    -2616.0            -2695.0 -2616.0       4038.525    4039.740   
2843491       -4.0             -278.0   -35.0      25421.985   25421.985   
2843492     -338.0             -668.0  -344.0      21876.300   21919.500   
2843494     -751.0             -782.0  -770.0      47029.500  935764.965   
2843495     -259.0             -453.0  -264.0      23556.195  656193.015   

                                      AMT_PAYMENT                          \
                   sum           mean         min         max         sum   
SK_ID_PREV                                                                  
1000001      68443.425   34221.712500    6404.310   62039.115   68443.425   
1000003      14854.050    4951.350000    4951.350    4951.350   14854.050   
1000004      33523.155    4789.022143    3391.110   13176.495   33523.155   
1000005     161735.310   14703.210000       2.790   14713.605  147021.705   
1000007      56234.025   11246.805000   11246.805   11246.805   56234.025   
...                ...            ...         ...         ...         ...   
2843490      16157.745    4039.436250    4038.525    4039.740   16157.745   
2843491     254219.850   25421.985000   25421.985   25421.985  254219.850   
2843492     262990.800   21915.900000   21876.300   21919.500  262990.800   
2843494     982794.465  491397.232500   47029.500  935764.965  982794.465   
2843495     797530.185  113932.883571   23556.195  656193.015  797530.185   

                                DPD        DBD  
                     mean      mean       mean  
SK_ID_PREV                                      
1000001      34221.712500  0.000000  16.000000  
1000003       4951.350000  0.000000  15.333333  
1000004       4789.022143  0.000000  26.714286  
1000005      13365.609545  0.363636   8.818182  
1000007      11246.805000  0.000000  16.800000  
...                   ...       ...        ...  
2843490       4039.436250  2.750000   0.000000  
2843491      25421.985000  0.000000  14.200000  
2843492      21915.900000  0.000000   6.250000  
2843494     491397.232500  0.000000  10.000000  
2843495     113932.883571  0.142857   4.000000  

[549020 rows x 16 columns]

In [37]:
ins_agg.columns = pd.Index([e[0] + '_' + e[1].upper() for e in ins_agg.columns.tolist()])


In [38]:
ins_agg.head()

,NUM_INSTALMENT_VERSION_NUNIQUE,NUM_INSTALMENT_NUMBER_MAX,DAYS_INSTALMENT_MIN,DAYS_INSTALMENT_MAX,DAYS_ENTRY_PAYMENT_MIN,DAYS_ENTRY_PAYMENT_MAX,AMT_INSTALMENT_MIN,AMT_INSTALMENT_MAX,AMT_INSTALMENT_SUM,AMT_INSTALMENT_MEAN,AMT_PAYMENT_MIN,AMT_PAYMENT_MAX,AMT_PAYMENT_SUM,AMT_PAYMENT_MEAN,DPD_MEAN,DBD_MEAN
SK_ID_PREV,,,,,,,,,,,,,,,,
1000001,2,2,-268.0,-238.0,-294.0,-244.0,6404.310,62039.115,68443.425,34221.712500,6404.310,62039.115,68443.425,34221.712500,0.000000,16.000000
1000003,1,3,-94.0,-34.0,-108.0,-49.0,4951.350,4951.350,14854.050,4951.350000,4951.350,4951.350,14854.050,4951.350000,0.000000,15.333333
1000004,2,7,-862.0,-682.0,-881.0,-695.0,3391.110,13176.495,33523.155,4789.022143,3391.110,13176.495,33523.155,4789.022143,0.000000,26.714286
1000005,1,10,-1688.0,-1418.0,-1687.0,-1433.0,14599.260,14713.605,161735.310,14703.210000,2.790,14713.605,147021.705,13365.609545,0.363636,8.818182
1000007,1,5,-123.0,-3.0,-143.0,-10.0,11246.805,11246.805,56234.025,11246.805000,11246.805,11246.805,56234.025,11246.805000,0.000000,16.800000


In [39]:
ins_agg.drop(['DAYS_INSTALMENT_MIN',
              'DAYS_INSTALMENT_MAX',
              'DAYS_ENTRY_PAYMENT_MIN',
              'DAYS_ENTRY_PAYMENT_MAX'],axis=1,inplace=True)

In [40]:
ins_agg['PAYMENT_PERC'] = ins_agg['AMT_PAYMENT_SUM'] / ins_agg['AMT_INSTALMENT_SUM']
ins_agg['PAYMENT_DIFF'] = ins_agg['AMT_INSTALMENT_SUM'] - ins_agg['AMT_PAYMENT_SUM']

In [41]:
ins_agg.head()

,NUM_INSTALMENT_VERSION_NUNIQUE,NUM_INSTALMENT_NUMBER_MAX,AMT_INSTALMENT_MIN,AMT_INSTALMENT_MAX,AMT_INSTALMENT_SUM,AMT_INSTALMENT_MEAN,AMT_PAYMENT_MIN,AMT_PAYMENT_MAX,AMT_PAYMENT_SUM,AMT_PAYMENT_MEAN,DPD_MEAN,DBD_MEAN,PAYMENT_PERC,PAYMENT_DIFF
SK_ID_PREV,,,,,,,,,,,,,,
1000001,2,2,6404.310,62039.115,68443.425,34221.712500,6404.310,62039.115,68443.425,34221.712500,0.000000,16.000000,1.000000,0.000
1000003,1,3,4951.350,4951.350,14854.050,4951.350000,4951.350,4951.350,14854.050,4951.350000,0.000000,15.333333,1.000000,0.000
1000004,2,7,3391.110,13176.495,33523.155,4789.022143,3391.110,13176.495,33523.155,4789.022143,0.000000,26.714286,1.000000,0.000
1000005,1,10,14599.260,14713.605,161735.310,14703.210000,2.790,14713.605,147021.705,13365.609545,0.363636,8.818182,0.909027,14713.605
1000007,1,5,11246.805,11246.805,56234.025,11246.805000,11246.805,11246.805,56234.025,11246.805000,0.000000,16.800000,1.000000,0.000


In [42]:
# count missings
nas = count_missings(ins_agg)
nas.head()

,Total,Percent
AMT_PAYMENT_MIN,33,0.006011
AMT_PAYMENT_MAX,33,0.006011
AMT_PAYMENT_MEAN,33,0.006011


In [43]:
ins_agg.to_csv('C:\\Users\\Dell V3400\\Downloads\\visualization\\dseb63_final_project_DP_dataset\\ins_agg_demo.csv')

#### 3.3. Credit card

In [44]:
card = pd.read_csv('C:\\Users\\Dell V3400\\Downloads\\visualization\\dseb63_final_project_DP_dataset\\dseb63_credit_card_balance.csv')

In [45]:
card

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,SK_ID_CURR
0,2582071,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.0,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0,87788
1,2582071,-82,16809.210,67500,0.0,0.0,0.0,0.0,3375.0,9000.0,9000.0,15488.685,16809.210,16809.210,0.0,0,0.0,0.0,18.0,Active,0,0,87788
2,2582071,-84,27577.890,67500,0.0,0.0,0.0,0.0,3375.0,4500.0,4500.0,26125.020,27577.890,27577.890,0.0,0,0.0,0.0,16.0,Active,0,0,87788
3,2582071,-7,65159.235,45000,0.0,0.0,0.0,0.0,2250.0,2250.0,2250.0,60301.170,65609.235,65609.235,0.0,0,0.0,0.0,63.0,Active,0,0,87788
4,2582071,-59,70475.850,67500,24750.0,24750.0,0.0,0.0,3375.0,4500.0,4500.0,63975.015,70475.850,70475.850,4.0,4,0.0,0.0,41.0,Active,0,0,87788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3227960,1551072,-1,0.000,202500,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0,259579
3227961,2339982,-1,0.000,45000,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.000,0.000,0.000,0.0,0,0.0,0.0,0.0,Active,0,0,220697
3227962,2720102,-1,0.000,225000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0,93573
3227963,1897864,-2,0.000,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0,11072


* FEATURE ENGINEERING

In [46]:
# logarithms
log_vars = ["AMT_BALANCE", "AMT_CREDIT_LIMIT_ACTUAL", "AMT_DRAWINGS_ATM_CURRENT", "AMT_DRAWINGS_CURRENT",
            "AMT_DRAWINGS_OTHER_CURRENT", "AMT_DRAWINGS_POS_CURRENT", "AMT_INST_MIN_REGULARITY",
            "AMT_PAYMENT_CURRENT", "AMT_PAYMENT_TOTAL_CURRENT", "AMT_RECEIVABLE_PRINCIPAL",
            "AMT_RECIVABLE", "AMT_TOTAL_RECEIVABLE"]
card = create_logs(card, log_vars, replace = True)

In [47]:
# dummy encodnig for factors
card = pd.get_dummies(card, columns= ['NAME_CONTRACT_STATUS'] )

In [48]:
card[card['NAME_CONTRACT_STATUS_Completed']].shape

(100031, 29)

In [49]:
# count missings
nas = count_missings(card)
nas

,Total,Percent
AMT_PAYMENT_CURRENT,620093,19.210029
AMT_DRAWINGS_OTHER_CURRENT,605754,18.765817
CNT_DRAWINGS_OTHER_CURRENT,605754,18.765817
AMT_DRAWINGS_POS_CURRENT,605754,18.765817
CNT_DRAWINGS_ATM_CURRENT,605754,18.765817
AMT_DRAWINGS_ATM_CURRENT,605754,18.765817
CNT_DRAWINGS_POS_CURRENT,605754,18.765817
AMT_INST_MIN_REGULARITY,264384,8.190423
CNT_INSTALMENT_MATURE_CUM,264384,8.190423


In [50]:
card['CREDIT_UTILIZATION'] = card['AMT_BALANCE'] - card['AMT_CREDIT_LIMIT_ACTUAL']
card['MIN_PAYMENT_VS_DRAWINGS'] = card['AMT_INST_MIN_REGULARITY'] - card['AMT_DRAWINGS_CURRENT']
card['PAYMENT_VS_DRAWINGS'] = card['AMT_PAYMENT_TOTAL_CURRENT'] - card['AMT_DRAWINGS_CURRENT']
card['PAYMENT_VS_TOTAL_RECEIVABLE'] = card['AMT_PAYMENT_TOTAL_CURRENT'] - card['AMT_TOTAL_RECEIVABLE']
card['PAYMENT_VS_BALANCE'] = card['AMT_PAYMENT_TOTAL_CURRENT'] - card['AMT_BALANCE']
card['PAYMENT_VS_MIN_INSTALLMENT'] = card['AMT_PAYMENT_TOTAL_CURRENT'] - card['AMT_INST_MIN_REGULARITY']
card['OVERDRAFT_AMOUNT'] = card['AMT_DRAWINGS_CURRENT'] - card['AMT_CREDIT_LIMIT_ACTUAL']
card['BALANCE_VS_TOTAL_RECEIVABLE'] = card['AMT_BALANCE'] - card['AMT_TOTAL_RECEIVABLE']
card['SUM_ALL_AMT_DRAWINGS'] = card[['AMT_DRAWINGS_ATM_CURRENT', 
                                     'AMT_DRAWINGS_CURRENT', 
                                     'AMT_DRAWINGS_OTHER_CURRENT', 
                                     'AMT_DRAWINGS_POS_CURRENT']].sum(axis=1)
card['SUM_ALL_CNT_DRAWINGS'] = card[['CNT_DRAWINGS_ATM_CURRENT', 
                                     'CNT_DRAWINGS_CURRENT', 
                                     'CNT_DRAWINGS_OTHER_CURRENT', 
                                     'CNT_DRAWINGS_POS_CURRENT']].sum(axis=1)
card['RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS'] = card['SUM_ALL_AMT_DRAWINGS'] / card['SUM_ALL_CNT_DRAWINGS']

In [51]:
grp = card.groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].agg('nunique').reset_index().rename(index=str, columns={'SK_ID_PREV': 'NUMBER_OF_LOANS'})
card = card.merge(grp, on=['SK_ID_CURR'], how='left')

In [52]:
grp = card.groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].nunique().reset_index().rename(index=str, columns={'SK_ID_PREV': 'NUMBER_OF_LOANS_PER_CUSTOMER'})
card = card.merge(grp, on=['SK_ID_CURR'], how='left')

In [53]:
grp = card.groupby(by=['SK_ID_CURR', 'SK_ID_PREV'])['CNT_INSTALMENT_MATURE_CUM'].max().reset_index().rename(index=str, columns={'CNT_INSTALMENT_MATURE_CUM': 'NUMBER_OF_INSTALMENTS'})
grp1 = grp.groupby(by=['SK_ID_CURR'])['NUMBER_OF_INSTALMENTS'].sum().reset_index().rename(index=str, columns={'NUMBER_OF_INSTALMENTS': 'TOTAL_INSTALMENTS'})
card = card.merge(grp1, on=['SK_ID_CURR'], how='left')

In [54]:
card['INSTALLMENTS_PER_LOAN'] = (card['TOTAL_INSTALMENTS'] / card['NUMBER_OF_LOANS_PER_CUSTOMER']).astype('uint32')

In [55]:
card['DPD'] = (card['SK_DPD'] > 0).astype(int)
grp = card.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['DPD'].sum().reset_index().rename(columns={'DPD': 'NUMBER_OF_DPD'})
grp1 = grp.groupby('SK_ID_CURR')['NUMBER_OF_DPD'].mean().reset_index().rename(columns={'NUMBER_OF_DPD': 'DPD_COUNT'})
card = card.merge(grp1, on=['SK_ID_CURR'], how='left')

In [56]:
def f(min_pay, total_pay):
    M = min_pay.tolist()
    T = total_pay.tolist()
    P = len(M)  # P: number of installments
    c = 0
    for i in range(len(M)):
        if T[i] < M[i]:
            c += 1
    return (100 * c) / P


grp = card.groupby(by=['SK_ID_CURR']).apply(lambda x: f(x.AMT_INST_MIN_REGULARITY, x.AMT_PAYMENT_CURRENT)).reset_index().rename(index=str, columns={0: 'PERCENTAGE_MIN_MISSED_PAYMENTS'})
card = card.merge(grp, on=['SK_ID_CURR'], how='left')

In [57]:
grp = card.groupby(by=['SK_ID_CURR'])['AMT_DRAWINGS_ATM_CURRENT'].sum().reset_index().rename(index=str, columns={'AMT_DRAWINGS_ATM_CURRENT': 'DRAWINGS_ATM'})
card = card.merge(grp, on=['SK_ID_CURR'], how='left')

In [58]:
grp = card.groupby(by=['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].sum().reset_index().rename(index=str, columns={'AMT_DRAWINGS_CURRENT': 'DRAWINGS_TOTAL'})
card = card.merge(grp, on=['SK_ID_CURR'], how='left')

In [59]:
card['CASH_CARD_RATIO'] = (card['DRAWINGS_ATM'] / card['DRAWINGS_TOTAL']) * 100
del card['DRAWINGS_ATM']
del card['DRAWINGS_TOTAL']

In [60]:
grp = card.groupby(by=['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].sum().reset_index().rename(index=str, columns={'AMT_DRAWINGS_CURRENT': 'TOTAL_DRAWINGS'})
card = card.merge(grp, on=['SK_ID_CURR'], how='left')

grp = card.groupby(by=['SK_ID_CURR'])['CNT_DRAWINGS_CURRENT'].sum().reset_index().rename(index=str, columns={'CNT_DRAWINGS_CURRENT': 'NUMBER_OF_DRAWINGS'})
card = card.merge(grp, on=['SK_ID_CURR'], how='left')

card['DRAWINGS_RATIO'] = (card['TOTAL_DRAWINGS'] / card['NUMBER_OF_DRAWINGS']) * 100
del card['TOTAL_DRAWINGS']
del card['NUMBER_OF_DRAWINGS']

In [61]:
grouped = card.groupby(['SK_ID_CURR', 'SK_ID_PREV', 'AMT_CREDIT_LIMIT_ACTUAL'])[['AMT_BALANCE']].max().reset_index()
grouped['CREDIT_CARD_BALANCE_RATIO'] = grouped['AMT_BALANCE'] / grouped['AMT_CREDIT_LIMIT_ACTUAL']
card = card.merge(grouped[['SK_ID_CURR', 'SK_ID_PREV', 'AMT_CREDIT_LIMIT_ACTUAL', 'CREDIT_CARD_BALANCE_RATIO']], on=['SK_ID_CURR', 'SK_ID_PREV', 'AMT_CREDIT_LIMIT_ACTUAL'], how='left')

card['INSTALMENTS_PER_LOAN'] = card['TOTAL_INSTALMENTS'] / card['NUMBER_OF_LOANS']

In [62]:
card

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,SK_ID_CURR,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed,CREDIT_UTILIZATION,MIN_PAYMENT_VS_DRAWINGS,PAYMENT_VS_DRAWINGS,PAYMENT_VS_TOTAL_RECEIVABLE,PAYMENT_VS_BALANCE,PAYMENT_VS_MIN_INSTALLMENT,OVERDRAFT_AMOUNT,BALANCE_VS_TOTAL_RECEIVABLE,SUM_ALL_AMT_DRAWINGS,SUM_ALL_CNT_DRAWINGS,RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS,NUMBER_OF_LOANS,NUMBER_OF_LOANS_PER_CUSTOMER,TOTAL_INSTALMENTS,INSTALLMENTS_PER_LOAN,DPD,DPD_COUNT,PERCENTAGE_MIN_MISSED_PAYMENTS,CASH_CARD_RATIO,DRAWINGS_RATIO,CREDIT_CARD_BALANCE_RATIO,INSTALMENTS_PER_LOAN
0,2582071,-1,11.066272,10.714440,7.719130,7.719130,0.0,0.0,7.719130,7.719130,7.719130,11.005030,11.080242,11.080242,1.0,1,0.0,0.0,69.0,0,0,87788,True,False,False,False,False,False,False,0.351832,0.000000,0.000000,-3.361112,-3.347142,0.000000,-2.995310,-0.013970,15.438260,2.0,7.719130,1,1,69.0,69,0,1.0,1.041667,97.883059,322.669937,1.035221,69.0
1,2582071,-82,9.729742,11.119898,0.000000,0.000000,0.0,0.0,8.124447,9.105091,9.105091,9.647930,9.729742,9.729742,0.0,0,0.0,0.0,18.0,0,0,87788,True,False,False,False,False,False,False,-1.390156,8.124447,9.105091,-0.624651,-0.624651,0.980644,-11.119898,0.000000,0.000000,0.0,NaN,1,1,69.0,69,0,1.0,1.041667,97.883059,322.669937,1.007000,69.0
2,2582071,-84,10.224806,11.119898,0.000000,0.000000,0.0,0.0,8.124447,8.412055,8.412055,10.170687,10.224806,10.224806,0.0,0,0.0,0.0,16.0,0,0,87788,True,False,False,False,False,False,False,-0.895092,8.124447,8.412055,-1.812751,-1.812751,0.287608,-11.119898,0.000000,0.000000,0.0,NaN,1,1,69.0,69,0,1.0,1.041667,97.883059,322.669937,1.007000,69.0
3,2582071,-7,11.084605,10.714440,0.000000,0.000000,0.0,0.0,7.719130,7.719130,7.719130,11.007123,11.091487,11.091487,0.0,0,0.0,0.0,63.0,0,0,87788,True,False,False,False,False,False,False,0.370165,7.719130,7.719130,-3.372357,-3.365475,0.000000,-10.714440,-0.006882,0.000000,0.0,NaN,1,1,69.0,69,0,1.0,1.041667,97.883059,322.669937,1.035221,69.0
4,2582071,-59,11.163040,11.119898,10.116621,10.116621,0.0,0.0,8.124447,8.412055,8.412055,11.066264,11.163040,11.163040,4.0,4,0.0,0.0,41.0,0,0,87788,True,False,False,False,False,False,False,0.043142,-1.992174,-1.704566,-2.750985,-2.750985,0.287608,-1.003277,0.000000,20.233242,8.0,2.529155,1,1,69.0,69,0,1.0,1.041667,97.883059,322.669937,1.007000,69.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3227960,1551072,-1,0.000000,12.218500,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0,NaN,NaN,0.0,0,0,259579,True,False,False,False,False,False,False,-12.218500,0.000000,0.000000,0.000000,0.000000,0.000000,-12.218500,0.000000,0.000000,0.0,NaN,1,1,0.0,0,0,0.0,0.000000,NaN,NaN,0.000000,0.0
3227961,2339982,-1,0.000000,10.714440,0.000000,0.000000,0.0,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.0,0,0.0,0.0,0.0,0,0,220697,True,False,False,False,False,False,False,-10.714440,0.000000,0.000000,0.000000,0.000000,0.000000,-10.714440,0.000000,0.000000,0.0,NaN,1,1,0.0,0,0,0.0,0.000000,NaN,NaN,0.000000,0.0
3227962,2720102,-1,0.000000,12.323860,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0,NaN,NaN,0.0,0,0,93573,True,False,False,False,False,False,False,-12.323860,0.000000,0.000000,0.000000,0.000000,0.000000,-12.323860,0.000000,0.000000,0.0,NaN,1,1,0.0,0,0,0.0,0.000000,NaN,NaN,

* AGGREGATIONS

In [63]:
card_agg = card.groupby('SK_ID_CURR').agg({
    'MONTHS_BALANCE': ["sum", "mean"],
    'AMT_BALANCE': ["sum", "mean", "min", "max", "std"],
    'AMT_CREDIT_LIMIT_ACTUAL': ["sum", "mean", "max", "min"],
    'AMT_DRAWINGS_ATM_CURRENT': ["sum", "mean", "min", "max"],
    'AMT_DRAWINGS_CURRENT': ["sum", "mean", "min", "max"],
    'AMT_DRAWINGS_OTHER_CURRENT': ["sum", "mean", "min", "max"],
    'AMT_DRAWINGS_POS_CURRENT': ["sum", "mean", "min", "max"],
    'AMT_INST_MIN_REGULARITY': ["sum", "mean", "min", "max"],
    'AMT_PAYMENT_CURRENT': ["sum", "mean", "min", "max"],
    'AMT_PAYMENT_TOTAL_CURRENT': ["sum", "mean", "min", "max"],
    'AMT_RECEIVABLE_PRINCIPAL': ["sum", "mean", "min", "max"],
    'AMT_RECIVABLE': ["sum", "mean", "min", "max"],
    'AMT_TOTAL_RECEIVABLE': ["sum", "mean", "min", "max"],
    'CNT_DRAWINGS_ATM_CURRENT': ["sum", "mean"],
    'CNT_DRAWINGS_CURRENT': ["sum", "mean", "max"],
    'CNT_DRAWINGS_OTHER_CURRENT': ["mean", "max"],
    'CNT_DRAWINGS_POS_CURRENT': ["sum", "mean", "max"],
    'CNT_INSTALMENT_MATURE_CUM': ["sum", "mean", "max", "min"],
    'SK_DPD': ["sum", "mean", "max"],
    'SK_DPD_DEF': ["sum", "mean", "max"],
    'NAME_CONTRACT_STATUS_Active': ["sum", "mean", "min", "max"],
    'NAME_CONTRACT_STATUS_Signed': ["sum", "mean", "min", "max"],
    'NAME_CONTRACT_STATUS_Completed': ["sum", "mean", "min", "max"],
    'NAME_CONTRACT_STATUS_Sent proposal': ["sum", "mean", "min", "max"],
    'NAME_CONTRACT_STATUS_Demand': ["sum", "mean", "min", "max"],
    'NAME_CONTRACT_STATUS_Refused': ["sum", "mean", "min", "max"],
    'NAME_CONTRACT_STATUS_Approved': ["sum", "mean", "min", "max"],
    'INSTALLMENTS_PER_LOAN': ["sum", "mean", "min", "max"],
    'NUMBER_OF_LOANS_PER_CUSTOMER': ["mean"],
    'DPD_COUNT': ["mean"],
    'PERCENTAGE_MIN_MISSED_PAYMENTS': ["mean"],
    'CASH_CARD_RATIO': ["mean"],
    'DRAWINGS_RATIO': ["mean"],
    'CREDIT_CARD_BALANCE_RATIO': ["mean", "max", "min"],
    'RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS': ['min', 'max', 'mean']
})

In [64]:
card_agg.columns = pd.Index(['CCB_' + e[0] + "_" + e[1].upper() for e in card_agg.columns.tolist()])

card_agg.reset_index(inplace = True)

In [65]:
card_agg

,SK_ID_CURR,CCB_MONTHS_BALANCE_SUM,CCB_MONTHS_BALANCE_MEAN,CCB_AMT_BALANCE_SUM,CCB_AMT_BALANCE_MEAN,CCB_AMT_BALANCE_MIN,CCB_AMT_BALANCE_MAX,CCB_AMT_BALANCE_STD,CCB_AMT_CREDIT_LIMIT_ACTUAL_SUM,CCB_AMT_CREDIT_LIMIT_ACTUAL_MEAN,CCB_AMT_CREDIT_LIMIT_ACTUAL_MAX,CCB_AMT_CREDIT_LIMIT_ACTUAL_MIN,CCB_AMT_DRAWINGS_ATM_CURRENT_SUM,CCB_AMT_DRAWINGS_ATM_CURRENT_MEAN,CCB_AMT_DRAWINGS_ATM_CURRENT_MIN,CCB_AMT_DRAWINGS_ATM_CURRENT_MAX,CCB_AMT_DRAWINGS_CURRENT_SUM,CCB_AMT_DRAWINGS_CURRENT_MEAN,CCB_AMT_DRAWINGS_CURRENT_MIN,CCB_AMT_DRAWINGS_CURRENT_MAX,CCB_AMT_DRAWINGS_OTHER_CURRENT_SUM,CCB_AMT_DRAWINGS_OTHER_CURRENT_MEAN,CCB_AMT_DRAWINGS_OTHER_CURRENT_MIN,CCB_AMT_DRAWINGS_OTHER_CURRENT_MAX,CCB_AMT_DRAWINGS_POS_CURRENT_SUM,CCB_AMT_DRAWINGS_POS_CURRENT_MEAN,CCB_AMT_DRAWINGS_POS_CURRENT_MIN,CCB_AMT_DRAWINGS_POS_CURRENT_MAX,CCB_AMT_INST_MIN_REGULARITY_SUM,CCB_AMT_INST_MIN_REGULARITY_MEAN,CCB_AMT_INST_MIN_REGULARITY_MIN,CCB_AMT_INST_MIN_REGULARITY_MAX,CCB_AMT_PAYMENT_CURRENT_SUM,CCB_AMT_PAYMENT_CURRENT_MEAN,CCB_AMT_PAYMENT_CURRENT_MIN,CCB_AMT_PAYMENT_CURRENT_MAX,CCB_AMT_PAYMENT_TOTAL_CURRENT_SUM,CCB_AMT_PAYMENT_TOTAL_CURRENT_MEAN,CCB_AMT_PAYMENT_TOTAL_CURRENT_MIN,CCB_AMT_PAYMENT_TOTAL_CURRENT_MAX,CCB_AMT_RECEIVABLE_PRINCIPAL_SUM,CCB_AMT_RECEIVABLE_PRINCIPAL_MEAN,CCB_AMT_RECEIVABLE_PRINCIPAL_MIN,CCB_AMT_RECEIVABLE_PRINCIPAL_MAX,CCB_AMT_RECIVABLE_SUM,CCB_AMT_RECIVABLE_MEAN,CCB_AMT_RECIVABLE_MIN,CCB_AMT_RECIVABLE_MAX,CCB_AMT_TOTAL_RECEIVABLE_SUM,CCB_AMT_TOTAL_RECEIVABLE_MEAN,CCB_AMT_TOTAL_RECEIVABLE_MIN,CCB_AMT_TOTAL_RECEIVABLE_MAX,CCB_CNT_DRAWINGS_ATM_CURRENT_SUM,CCB_CNT_DRAWINGS_ATM_CURRENT_MEAN,CCB_CNT_DRAWINGS_CURRENT_SUM,CCB_CNT_DRAWINGS_CURRENT_MEAN,CCB_CNT_DRAWINGS_CURRENT_MAX,CCB_CNT_DRAWINGS_OTHER_CURRENT_MEAN,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX,CCB_CNT_DRAWINGS_POS_CURRENT_SUM,CCB_CNT_DRAWINGS_POS_CURRENT_MEAN,CCB_CNT_DRAWINGS_POS_CURRENT_MAX,CCB_CNT_INSTALMENT_MATURE_CUM_SUM,CCB_CNT_INSTALMENT_MATURE_CUM_MEAN,CCB_CNT_INSTALMENT_MATURE_CUM_MAX,CCB_CNT_INSTALMENT_MATURE_CUM_MIN,CCB_SK_DPD_SUM,CCB_SK_DPD_MEAN,CCB_SK_DPD_MAX,CCB_SK_DPD_DEF_SUM,CCB_SK_DPD_DEF_MEAN,CCB_SK_DPD_DEF_MAX,CCB_NAME_CONTRACT_STATUS_Active_SUM,CCB_NAME_CONTRACT_STATUS_Active_MEAN,CCB_NAME_CONTRACT_STATUS_Active_MIN,CCB_NAME_CONTRACT_STATUS_Active_MAX,CCB_NAME_CONTRACT_STATUS_Signed_SUM,CCB_NAME_CONTRACT_STATUS_Signed_MEAN,CCB_NAME_CONTRACT_STATUS_Signed_MIN,CCB_NAME_CONTRACT_STATUS_Signed_MAX,CCB_NAME_CONTRACT_STATUS_Completed_SUM,CCB_NAME_CONTRACT_STATUS_Completed_MEAN,CCB_NAME_CONTRACT_STATUS_Completed_MIN,CCB_NAME_CONTRACT_STATUS_Completed_MAX,CCB_NAME_CONTRACT_STATUS_Sent proposal_SUM,CCB_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CCB_NAME_CONTRACT_STATUS_Sent proposal_MIN,CCB_NAME_CONTRACT_STATUS_Sent proposal_MAX,CCB_NAME_CONTRACT_STATUS_Demand_SUM,CCB_NAME_CONTRACT_STATUS_Demand_MEAN,CCB_NAME_CONTRACT_STATUS_Demand_MIN,CCB_NAME_CONTRACT_STATUS_Demand_MAX,CCB_NAME_CONTRACT_STATUS_Refused_SUM,CCB_NAME_CONTRACT_STATUS_Refused_MEAN,CCB_NAME_CONTRACT_STATUS_Refused_MIN,CCB_NAME_CONTRACT_STATUS_Refused_MAX,CCB_NAME_CONTRACT_STATUS_Approved_SUM,CCB_NAME_CONTRACT_STATUS_Approved_MEAN,CCB_NAME_CONTRACT_STATUS_Approved_MIN,CCB_NAME_CONTRACT_STATUS_Approved_MAX,CCB_INSTALLMENTS_PER_LOAN_SUM,CCB_INSTALLMENTS_PER_LOAN_MEAN,CCB_INSTALLMENTS_PER_LOAN_MIN,CCB_INSTALLMENTS_PER_LOAN_MAX,CCB_NUMBER_OF_LOANS_PER_CUSTOMER_MEAN,CCB_DPD_COUNT_MEAN,CCB_PERCENTAGE_MIN_MISSED_PAYMENTS_MEAN,CCB_CASH_CARD_RATIO_MEAN,CCB_DRAWINGS_RATIO_MEAN,CCB_CREDIT_CARD_BALANCE_RATIO_MEAN,CCB_CREDIT_CARD_BALANCE_RATIO_MAX,CCB_CREDIT_CARD_BALANCE_RATIO_MIN,CCB_RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS_MIN,CCB_RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS_MAX,CCB_RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS_MEAN
0,0,-36,-4.5,28.377469,3.547184,0.000000,10.506995,4.927492,96.805741,12.100718,12.100718,12.100718,0.000000,0.000000,0.0,0.000000,47.420183,5.927523,0.0,10.936458,0.000000,0.000000,0.0,0.000000,47.420183,5.927523,0.0,10.936458,7.719130,0.964891,0.0,7.719130,57.836401,8.262343,6.192506,11.059990,57.451956,7.18

In [66]:
pos_agg.to_csv('C:\\Users\\Dell V3400\\Downloads\\visualization\\dseb63_final_project_DP_dataset\\card_agg_demo.csv')